In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tqdm as tqdm

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score

In [3]:
from keras.models import Model
from keras.applications import Xception

In [4]:
img_size = [299,299]

In [5]:
training_data = 'rps'
test_data = 'rps-test-set'

In [6]:
xcep = Xception(input_shape = img_size+ [3], weights = 'imagenet', include_top=False)

In [7]:
for x in xcep.layers:
    x.trainable = False

In [8]:
from glob import glob
classes = glob('rps/*')
classes

['rps/paper', 'rps/scissors', 'rps/rock']

In [9]:
Flayer = tf.keras.layers.Flatten()(xcep.output)

In [10]:
FClayer = tf.keras.layers.Dense(250, activation = 'relu')(Flayer)

In [11]:
Olayer = tf.keras.layers.Dense(len(classes), activation = 'softmax')(FClayer)

In [12]:
DD = Model(inputs = xcep.input, outputs = Olayer)

In [13]:
DD.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [14]:
DD.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [15]:
train_data_generator = ImageDataGenerator(rescale = 1./255,
                                          horizontal_flip=True,
                                          shear_range=0.2,
                                          zoom_range=0.2)

training_data = train_data_generator.flow_from_directory('rps',
                                                         target_size = (299,299),
                                                         batch_size = 32,
                                                         class_mode = 'categorical')

Found 2514 images belonging to 3 classes.


In [16]:
test_data_generator = ImageDataGenerator(rescale = 1./255)

testing_data = test_data_generator.flow_from_directory('rps-test-set',
                                                    target_size = (299,299),
                                                    batch_size = 32,
                                                    class_mode = 'categorical')

Found 372 images belonging to 3 classes.


In [ ]:
J = DD.fit_generator(training_data,
                     validation_data=testing_data,
                     epochs= 6,
                     validation_steps=len(testing_data),
                     steps_per_epoch = len(training_data))

/home/cyber-mysterio/anaconda3/envs/ultron/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/6
79/79 [==============================] - 470s 6s/step - loss: 10.5733 - accuracy: 0.7522 - val_loss: 2.7115 - val_accuracy: 0.8522
Epoch 2/6
 2/79 [..............................] - ETA: 4:59 - loss: 2.3149e-05 - accuracy: 1.0000 

In [ ]:
plt.plot(J.history['loss'], label = 'Train Loss')
plt.plot(J.history['val_loss'], label = 'Val Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(J.history['accuracy'], label = 'Train Accuracy')
plt.plot(J.history['val_accuracy'], label = 'Val Accuracy')
plt.legend()
plt.show()

In [ ]:
DD.save('Xception RPS.h5')

In [ ]:
from keras.preprocessing import image
img = image.load_img('New Folder/paper07-106.png', target_size = (299,299))

In [ ]:
img

In [ ]:
imgA = image.img_to_array(img)

In [ ]:
imgD = np.expand_dims(imgA, axis=0)

In [ ]:
img_data = tf.keras.applications.xception.preprocess_input(imgD)

In [ ]:
pred = DD.predict(img_data)

In [ ]:
pred